In [ ]:
import nnabla as nn

import nnabla.functions as F
import nnabla.parametric_functions as PF
import nnabla.solvers as S
from nnabla.monitor import tile_images

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
lr = 1e-4
lam1 = 1e-2
lam2 = 1e-2

In [ ]:
def G(z, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("G"):  # Parameter scope can be nested
        h = z
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last_layer"):
            X = F.sigmoid(PF.affine(h, X_dim))
    return X

In [ ]:
def E(X, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("E"):  # Parameter scope can be nested
        h = X
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("last_layer"):
            z = PF.affine(h, Z_dim)
    return z

In [ ]:
def D(X, hidden=[128, 128]):
    hs = []
    with nn.parameter_scope("D"):  # Parameter scope can be nested
        h = X
        for hid, hsize in enumerate(hidden):
            with nn.parameter_scope("affine{}".format(hid + 1)):
                h = F.relu(PF.affine(h, hsize))
                hs.append(h)
        with nn.parameter_scope("classifier"):
            y = F.sigmoid(PF.affine(h, 1))
    return y

In [ ]:
G_solver = S.Adam(lr)
with nn.parameter_scope("G"):
    G_solver.set_parameters(nn.get_parameters())
    
D_solver = S.Adam(lr)
with nn.parameter_scope("D"):
    D_solver.set_parameters(nn.get_parameters())
    
E_solver = S.Adam(lr)
with nn.parameter_scope("E"):
    E_solver.set_parameters(nn.get_parameters())

In [ ]:
def reset_grad():
    G_solver.zero_grad()
    D_solver.zero_grad()
    E_solver.zero_grad()

In [ ]:
def sample_X(size, with_y=False):
    X, y = mnist.train.next_batch(size)
    X = nn.Variable.from_numpy_array(X)
    
    if with_y:
        y = np.argmax(y, axis=1).astype(np.int)
        y = nn.Variable.from_numpy_array(y)
        return X, y
    else:
        return X

In [ ]:
def show16(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.show()

In [ ]:
for it in range(1000000):
    # Discriminator Update
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    X = sample_X(mb_size)

    G_sample = G(z)
    D_real = D(X)
    D_fake = D(G_sample)

    D_loss = -F.mean(F.log(D_real + 1e-8) + F.log(1 - D_fake + 1e-8))

    D_loss.forward()
    D_loss.backward() 
    D_solver.update()

    reset_grad()

    # Generator Update
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    X = sample_X(mb_size)
    
    G_sample = G(z)
    G_sample_reg = G(E(X))
    D_fake = D(G_sample)
    D_reg = D(G_sample_reg)


    mse = F.sum((X - G_sample_reg)**2, axis=1, keepdims=True)
    reg = F.mean(lam1 * mse + lam2 * F.log(D_reg + 1e-8))
    G_loss = -F.mean(F.log(D_fake + 1e-8)) + reg

    G_loss.forward()
    G_loss.backward()
    G_solver.update()

    reset_grad()
    
    # Encoder Update
    z = nn.Variable.from_numpy_array(np.random.randn(mb_size, Z_dim))
    X = sample_X(mb_size)
    
    G_sample_reg = G(E(X))
    D_reg = D(G_sample_reg)
    
    mse = F.sum((X - G_sample_reg)**2, axis=1, keepdims=True)
    E_loss = F.mean(lam1 * mse + lam2 * F.log(D_reg + 1e-8))
    
    E_loss.forward()
    E_loss.backward()
    E_solver.update()

    reset_grad()
    
    # Generate and Show Samples 
    if it % 1000 == 0:
        idx = np.random.randint(0, 10)
        print('Step: {}, D_loss: {}, E_loss: {}, G_loss: {}'.format(it, D_loss.d, E_loss.d, G_loss.d))
        
        samples = G(z).d[:16]
        show16(samples)